Here do also the t-test for the results of the different models by metric.

In [ ]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import glob
from utils import display_results, display_structure


In [ ]:
# import any json file from the results folders

file = "./results/targCls_2_music_openl3_env_voice_True_TotCls_5_dropNs_True_rep_5_fold_5.json"

with open(file) as f:
    data = json.load(f)
    print(file)
    display_results(data)
    #print(data["config"]["classifications"].keys())


In [ ]:
columns = ["Embeddings" ,"Target F1", "Avg. secondary F1", "Avg. R2 emotions", "Avg. r emotions", "Avg. R2 mid-level", "Avg. r mid-level"]

lvl_1 = ["Binary", "Ternary"]

lvl_2 = ["Voice", "No voice"]

dict_of_dfs = {k1: {k2: pd.DataFrame(columns=columns) for k2 in lvl_2} for k1 in lvl_1}

for fn in glob.glob("./results/*.json"):
    
    with open(fn) as f:
        results = json.load(f)

    # aggregate secondary f1 scores
    f1_secondary_means = [v['mean'] for k,v in results["f1"].items() if k != 'target']
    f1_secondary_stds = [v['std'] for k,v in results["f1"].items() if k != 'target']
    results["average"]["secondary f1"] = { "mean": np.mean(f1_secondary_means), "std": np.mean(f1_secondary_stds)}

    first_level = "Binary" if len(results["config"]["classifications"]["target"]) == 2 else "Ternary"
    second_level = "Voice" if results["config"]["voice"] else "No voice"

    row = {
        "Embeddings": results["config"]["which_embeddings"],
        "Target F1": f'{results["f1"]["target"]["mean"]:.2f} ± {results["f1"]["target"]["std"]:.2f}',
        "Avg. secondary F1": f'{results["average"]["secondary f1"]["mean"]:.2f} ± {results["average"]["secondary f1"]["std"]:.2f}',
        "Avg. R2 emotions": f'{results["average"]["emo_r2"]["mean"]:.2f} ± {results["average"]["emo_r2"]["std"]:.2f}',
        "Avg. r emotions": f'{results["average"]["emo_pears"]["mean"]:.2f} ± {results["average"]["emo_pears"]["std"]:.2f}',
        "Avg. R2 mid-level": f'{results["average"]["mid_r2"]["mean"]:.2f} ± {results["average"]["mid_r2"]["std"]:.2f}',
        "Avg. r mid-level": f'{results["average"]["mid_pears"]["mean"]:.2f} ± {results["average"]["mid_pears"]["std"]:.2f}',
        

    }

    df = dict_of_dfs[first_level][second_level].copy()
    df = pd.concat([df, pd.DataFrame(row, index=[0])], ignore_index=True)
    dict_of_dfs[first_level][second_level] = df




In [ ]:
for first_level in lvl_1:
    for second_level in lvl_2:
        print(f"{first_level} - {second_level}")
        display(dict_of_dfs[first_level][second_level])